In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=430c9565ab9861780167adb70019ebb461eab81ea8ee7199ba4fde78d9aec1dd
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [4]:
import feedparser

In [5]:
import transformers, requests

In [8]:
from transformers import pipeline

pipe = pipeline(task="text-classification", model="ProsusAI/finbert")

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
print(pipe("Stocks rallied today."))

[{'label': 'positive', 'score': 0.8962162137031555}]


In [27]:
ticker = 'GC=F'
keyword = 'gold'

#### Yahoo Finance

In [28]:
rss_url = f'https://finance.yahoo.com/rss/headline?s={ticker}'

In [29]:
feed = feedparser.parse(rss_url)

In [30]:
total_score = 0
num_articles = 0

In [31]:
for i, entry in enumerate(feed.entries):
    if keyword.lower() not in entry.summary.lower():
        continue
    
    print(f"Title: {entry.title}")
    print(f"Link: {entry.link}")
    print(f"Published: {entry.published}")
    print(f"Summary: {entry.summary}")
    
    sentiment = pipe(entry.summary)[0]
    
    print(f"Sentiment {sentiment['label']}, Score: {sentiment['score']}")
    print('-'*20)
    
    num_articles += 1

    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']

Title: Investors should 'go for gold' as Fed rate cut looms, Goldman says
Link: https://finance.yahoo.com/news/investors-should-go-for-gold-as-fed-rate-cut-looms-goldman-says-155551358.html?.tsrc=rss
Published: Tue, 03 Sep 2024 15:55:51 +0000
Summary: Goldman Sachs analysts say gold is the firm's preferred commodity as it sits near its two-week high in price.
Sentiment positive, Score: 0.7551811933517456
--------------------
Title: How gold has become an 'uncertainty hedge'
Link: https://finance.yahoo.com/video/gold-become-uncertainty-hedge-110000678.html?.tsrc=rss
Published: Mon, 02 Sep 2024 15:00:00 +0000
Summary: Tom Bruni, Head of Market Research at StockTwits, joined Stocks In Translation to explore the benefits of including gold (GC=F) in your investment portfolio as a safety net. He noted, “There's a ton of volatility there. So it attracts a lot of traders.” Listen to the full episode here, or wherever you get your podcasts. For expert insight and the latest market action, click

In [32]:
final_score = total_score / num_articles

In [33]:
print(f"Overall Sentiment: {'Positive' if final_score >= 0.15 else 'Negative' if final_score <= -0.15 else 'Neutral'} {final_score} ")

Overall Sentiment: Neutral 0.031879544258117676 


### NewsAPI

In [35]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
NewsAPI = user_secrets.get_secret("NewsAPI")

In [37]:
keyword = 'gold'
date = '2024-09-02'

In [38]:
url = (f'https://newsapi.org/v2/everything?q={keyword}&from={date}&sortBy=popularity&apiKey={NewsAPI}')

In [39]:
response = requests.get(url)

In [42]:
articles = response.json()['articles']
articles = [article for article in articles if keyword.lower() in article['title'].lower() or keyword.lower() in article['description'].lower()]

In [44]:
total_score = 0
num_articles = 0

In [47]:
for i, article in enumerate(articles):
    if keyword.lower() not in article["content"].lower():
        continue
    
    print(f"Title: {article['title']}")
    print(f"Link: {article['url']}")
    print(f"Published: {article['description']}")
    
    sentiment = pipe(article['content'])[0]
    
    print(f"Sentiment {sentiment['label']}, Score: {sentiment['score']}")
    print('-'*20)
    
    num_articles += 1

    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']

Title: Pregnant archer Grinham wins Paralympic gold
Link: https://www.bbc.com/sport/archery/articles/cm23px4ej4wo
Published: Pregnant Para-archer Jodie Grinham and Nathan MacQueen win gold for Great Britain in the mixed team compound final.
Sentiment neutral, Score: 0.6428490877151489
--------------------
Title: GB's Challis dominant to claim backstroke gold
Link: https://www.bbc.com/sport/swimming/articles/c8dp8yvl6y6o
Published: Great Britain's Ellie Challis wins gold with a dominant performance in the women's 50m backstroke final.
Sentiment positive, Score: 0.7158695459365845
--------------------
Title: Inspired McGuire wins boccia gold for GB
Link: https://www.bbc.com/sport/articles/c79w4v771z5o
Published: Britain's Stephen McGuire wins Paralympic boccia gold, defeating Colombia's Edilson Chica to win the BC4 men's individual title.
Sentiment neutral, Score: 0.6288122534751892
--------------------
Title: Ellis wins GB's first Paralympic gold of day five
Link: https://www.bbc.com/sp

In [48]:
final_score = total_score / num_articles

In [49]:
print(f"Overall Sentiment: {'Positive' if final_score >= 0.15 else 'Negative' if final_score <= -0.15 else 'Neutral'} {final_score} ")

Overall Sentiment: Neutral 0.0477246363957723 
